In [115]:
import json
import requests
import numpy as np
with open('data1.json', 'r', encoding='utf-8') as json_file:
    inp = json.load(json_file)

In [116]:
url = "http://reg.gst.gov.in/registration/"
response = requests.get(url)
cookie=response.headers['Set-Cookie'].split(';')[0]

In [121]:
def create_captcha1(cookie):
    url = f"https://reg.gst.gov.in/services/captcha?rnd={np.random.random()}"
    headers = {
        "Host": "reg.gst.gov.in",
        "Cookie": cookie,
        "Accept": "image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8",
        "Referer": "https://reg.gst.gov.in/registration/",
        }
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open("captcha.png", "wb") as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)
        print("Captcha image saved as captcha.png")
    else:
        print("Failed to fetch captcha image")
    return response.headers['Set-Cookie'].split('; ')[0]

In [122]:
captchacookie=create_captcha1(cookie)

Captcha image saved as captcha.png


In [123]:
captcha="489079"

In [124]:

url = "https://reg.gst.gov.in/registration/api/submit/sendotp"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": f"{cookie}; {captchacookie}",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://reg.gst.gov.in"
}
data={'applnType': 'APLRG',
 'pt': 'P',
 'stCd': inp['opdtls'][0]['rsad']['stcd'],
 'dtCd': inp['opdtls'][0]['rsad']['dst'],
 'lgBzName': inp['opdtls'][0]['fn']+" "+inp['opdtls'][0]['mn']+" "+inp['opdtls'][0]['ln'],
 'id': inp['opdtls'][0]['pan']['num'],
 'email': inp['opdtls'][0]['em'],
 'mbno': inp['opdtls'][0]['mbno'],
 'captcha': captcha}
response = requests.post(url, headers=headers, data=json.dumps(data))

print("Response Status Code:", response.status_code)
print("Response Content:")
if 'invalid' not in json.loads(response.text)['message']:
    print(response.text)


Response Status Code: 200
Response Content:
{"stscd":"SUCCESS","message":"OTP sent successfully","errors":""}


In [125]:
del data['captcha']
data["smsOtp"]="571356"
data["emailOtp"]="997873"

In [126]:
import requests
import json

url = "https://reg.gst.gov.in/registration/api/submit/submitotp"

headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": f"{cookie}; {captchacookie}",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://reg.gst.gov.in"
}
response = requests.post(url, headers=headers, data=json.dumps(data))
trn = json.loads(response.text)['trn']


In [127]:
url = f"https://reg.gst.gov.in/services/captcha?rnd={np.random.random()}"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": cookie,
    "Accept": "image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8",
    "Referer": "https://reg.gst.gov.in/registration/",
    }
response = requests.get(url, headers=headers, stream=True)
if response.status_code == 200:
    with open("captcha1.png", "wb") as f:
        for chunk in response.iter_content(chunk_size=128):
            f.write(chunk)
    print("Captcha image saved as captcha1.png")
else:
    print("Failed to fetch captcha image")
captchacookie = response.headers['Set-Cookie'].split('; ')[0]

Captcha image saved as captcha1.png


In [128]:
captcha = "450438"

In [129]:
url = "https://reg.gst.gov.in/registration/api/submit/sendtrnotp"

headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": f"{cookie}; {captchacookie}",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://reg.gst.gov.in"
}
data={"trn":f"{trn}","captcha":captcha}
response = requests.post(url, headers=headers, data=json.dumps(data))


In [130]:
smsotp = "996289"

In [131]:
url = "https://reg.gst.gov.in/registration/api/submit/submittrnotp"

headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": f"{cookie}",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://reg.gst.gov.in"
}
data={"trn":f"{trn}","smsOtp": smsotp}
response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.text)
auth_token=response.headers['set-cookie'].split(';')[0].split('=')[1]

{"stscd":"SUCCESS","message":"New Registration","errors":"REG1011"}


In [132]:
new_cookie=f"{cookie};AuthToken={auth_token};"

In [133]:
url = "https://reg.gst.gov.in/registration/api/get/ExstPanDtls"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": cookie,
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://reg.gst.gov.in",
 }
data1 = {
    "pan": inp['opdtls'][0]['pan']['num'],
    "stateCode": inp['opdtls'][0]['rsad']['stcd']
}
response = requests.post(url, headers=headers, data=json.dumps(data1))

print(response.text)


{"pan":"BUTPJ7345F","panDate":"Pan date not available","regDtls":[]}


In [134]:
pandate=json.loads(response.text)['panDate']

In [135]:
url = "https://reg.gst.gov.in/registration/auth/api/myapp"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": new_cookie,
    }
response = requests.get(url, headers=headers)
print(response.text)
draft_id=eval(response.text)[0]['draft_id']
inp['aplCd']=draft_id


[{"creation_dt":"08/09/2023","form_no":"GST REG-01","expiry_dt":"23/09/2023","form_desc":"Application for New Registration","draft_id":48952576,"version_id":1,"state_cd":"29","status":"Draft","appcd":"APLRG"}]


In [136]:
#to upload photograph of promoter
url = "https://reg.gst.gov.in/document"
headers = {
    "Cookie": new_cookie,
    "X-Requested-With": "XMLHttpRequest",
    "Origin": "https://reg.gst.gov.in",
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/promoters"
}

files = {
    "upfile": ("WhatsApp_Image_2023_08_09_at_8.11.41_PM__1__11zon.jpeg", open(r"/home/pratik/Downloads/gst_kv/WhatsApp_Image_2023_08_09_at_8.11.41_PM__1__11zon.jpeg", "rb"), "image/jpeg"),
}

data1 = {
    "ty": "PHOT",
    "applnid": draft_id
}

response = requests.post(url, headers=headers, files=files, data=data1)
print("Response Content:", response.text)
prom_photo=json.loads(response.text)
inp['opdtls'][0]['dcupdtls']=[prom_photo]
inp['asgdtls'][0]['dcupdtls']=[prom_photo]

Response Content: {"id":107027179,"ty":"PHOT","ct":"image/jpeg","hash":"ccb45941ebdce11e6d5ef25b23c247d21551be69a658f271ce7f1013fde56616"}


In [137]:
inp

{'aplCd': 48952576,
 'rgdtls': {'aplst': 'DFT',
  'aplty': 'APLRG',
  'duedt': '23/09/2023',
  'moddt': '08/09/2023',
  'mbno': '9481829401',
  'em': 'kashvijain19@gmail.com'},
 'bzdtls': {'bzdtlsbz': {'addltrdnms': [],
   'regCategory': [],
   'rslibrg': 'DSEP',
   'iscasdl': 'N',
   'isopcmp': 'N',
   'lgnmbzpan': 'KASHVI DILEEP  JAIN',
   'pan': {'num': 'BUTPJ7345F',
    'iv': 'V',
    'panDate': 'Pan date not available'},
   'stcd': '29',
   'dst': 'KABNU',
   'trdnm': 'highheelsnthreads',
   'cobz': 'PRO',
   'cmbzdt': '08/09/2023',
   'librgdt': '08/09/2023',
   'otherRsLib': '',
   'sezDtls': {}},
  'exrgdtls': []},
 'decdtls': {'dt': '08/09/2023'},
 'secToken': '292300186862TRN',
 'map': {'opdtls': [{'eid': 1,
    'lt': '12.951735',
    'lg': '77.568406',
    'eloc': 'E71XNI',
    'city': 'Bengaluru',
    'geoCodeLevel': 'Building',
    'nrbyLdMrk': None,
    'locSubloc': 'Sankarapuram',
    'feedback': {'locSubloc': True,
     'st': True,
     'bnm': True,
     'bno': True,
  

In [138]:
# #auth signatory letter of auth file upload
    
# url = "https://reg.gst.gov.in/document"
# headers = {
#     "Cookie": new_cookie,
#     "Origin": "https://reg.gst.gov.in",
#     "Referer": "https://reg.gst.gov.in/registration/auth/newappl/authsignatory"
#     }

# files = {
#     "upfile": ("GST-Declaration-Authorised-Signatory_HIGH (1).pdf", open(r"gst_vdp/GST-Declaration-Authorised-Signatory_HIGH (1).pdf", "rb"), "application/pdf"),
# }

# data1 = {
#     "ty": "LOAU",
#     #"ty":"CRBC" if Copy of resolution passed by BoD / Managing Committee
#     "applnid": draft_id
# }

# response = requests.post(url, headers=headers, files=files, data=data1)
# print("Response Content:", response.text)
# auth_sign_att1=json.loads(response.text)


In [139]:
# #auth signatory photo file upload
# #limit file size to 100kb
# url = "https://reg.gst.gov.in/document"
# headers = {
#     "Cookie": new_cookie,
#     "Origin": "https://reg.gst.gov.in",
#     "Referer": "https://reg.gst.gov.in/registration/auth/newappl/authsignatory"
#     }
# files = {
#     "upfile": (r"kashvi_pic.jpeg", open(r"gst_vdp/kashvi_pic.jpeg", "rb"), "image/jpeg"),
# }
# data1 = {
#     "ty": "PHOT",
#     "applnid": draft_id
# }
# response = requests.post(url, headers=headers, files=files, data=data1)
# print("Response Content:", response.text)
# auth_sign_att2=json.loads(response.text)


In [140]:
# inp['asgdtls'][0]['dcupdtls']=[auth_sign_att1,auth_sign_att2]


In [141]:
# sourcery skip: avoid-builtin-shadow
url = "https://reg.gst.gov.in/documenthb/upload"
headers = {
    "Cookie": new_cookie,
    "Origin": "https://reg.gst.gov.in",
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/business/place"
}

files = {
    "upfile": ("WhatsApp_Image_2023_08_31_at_1.52.20_PM (1).jpeg", open(r"/home/pratik/Downloads/gst_kv/WhatsApp_Image_2023_08_31_at_1.52.20_PM (1).jpeg", "rb"), "image/jpeg"),
}
data1 = {
    "ty": "ELCB",
    "applnid": draft_id
}

response = requests.post(url, headers=headers, files=files, data=data1)

print("Response Status Code:", response.status_code)
print("Response Content:", response.text)
id=json.loads(response.text)['id']
url = f"https://reg.gst.gov.in/documenthb/docstatus/{id}"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": new_cookie,
    "Accept": "application/json, text/plain, */*",
    "At": auth_token,
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/business/place",
   }
response = requests.get(url, headers=headers)
print("Response Content:", response.text)
ppz_doc_att1=json.loads(response.text)


Response Status Code: 200
Response Content: {"status":"WAIT","id":"2023092940923"}
Response Content: {"id":"2023092940923","ty":"ELCB","ct":"image/jpeg","docName":"WhatsApp_Image_2023_08_31_at_1.52.20_PM (1).jpeg","hash":"7167284b4987aaa4cf9a3f53c8bcf7723ad8bc0a4b04950710cf4deff688647a"}


In [143]:
# sourcery skip: avoid-builtin-shadow
url = "https://reg.gst.gov.in/documenthb/upload"
headers = {
    "Cookie": new_cookie,
    "Origin": "https://reg.gst.gov.in",
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/business/place"
}
files = {
    "upfile": ("noc.pdf", open(r"/home/pratik/Downloads/gst_kv/noc.pdf", "rb"), "application/pdf"),
}
data1 = {
    "ty": "CNLR",
    "applnid": draft_id
}
response = requests.post(url, headers=headers, files=files, data=data1)
print("Response Content:", response.text)
id=json.loads(response.text)['id']
url = f"https://reg.gst.gov.in/documenthb/docstatus/{id}"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": new_cookie,
    "Accept": "application/json, text/plain, */*",
    "At": auth_token,
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/business/place",
   }
response = requests.get(url, headers=headers)

print("Response Status Code:", response.status_code)
print("Response Content:", response.text)
ppz_doc_att2=json.loads(response.text)


Response Content: {"status":"WAIT","id":"2023092940943"}
Response Status Code: 200
Response Content: {"id":"2023092940943","ty":"CNLR","ct":"application/pdf","docName":"noc.pdf","hash":"eb308e0b98d454f446b8bc1a3310416ec29ac1c712792c53440cb8e534b4d2e8"}


In [144]:
inp['ppbzdtls']['dcupdtls']=[ppz_doc_att1,ppz_doc_att2]
inp['secToken']=trn
inp['stsidtls']={}
inp['adhrflag']="Y"
inp['asgdtls'][0]['aadhaarflag']="Y"
inp['opdtls'][0]['aadhaarflag']="Y"


In [145]:
inp

{'aplCd': 48952576,
 'rgdtls': {'aplst': 'DFT',
  'aplty': 'APLRG',
  'duedt': '23/09/2023',
  'moddt': '08/09/2023',
  'mbno': '9481829401',
  'em': 'kashvijain19@gmail.com'},
 'bzdtls': {'bzdtlsbz': {'addltrdnms': [],
   'regCategory': [],
   'rslibrg': 'DSEP',
   'iscasdl': 'N',
   'isopcmp': 'N',
   'lgnmbzpan': 'KASHVI DILEEP  JAIN',
   'pan': {'num': 'BUTPJ7345F',
    'iv': 'V',
    'panDate': 'Pan date not available'},
   'stcd': '29',
   'dst': 'KABNU',
   'trdnm': 'highheelsnthreads',
   'cobz': 'PRO',
   'cmbzdt': '08/09/2023',
   'librgdt': '08/09/2023',
   'otherRsLib': '',
   'sezDtls': {}},
  'exrgdtls': []},
 'decdtls': {'dt': '08/09/2023'},
 'secToken': '292300186899TRN',
 'map': {'opdtls': [{'eid': 1,
    'lt': '12.951735',
    'lg': '77.568406',
    'eloc': 'E71XNI',
    'city': 'Bengaluru',
    'geoCodeLevel': 'Building',
    'nrbyLdMrk': None,
    'locSubloc': 'Sankarapuram',
    'feedback': {'locSubloc': True,
     'st': True,
     'bnm': True,
     'bno': True,
  

In [146]:
from datetime import date

# Get today's date
# today = date.today()

# Format it as a string in ddmmyyyy format
formatted_date = date.today().strftime("%d/%m/%Y")

# Print the formatted date
print(formatted_date)


08/09/2023


In [147]:
inp['decdtls']={'asdes': inp['opdtls'][0]['dg'], 'pl': inp['opdtls'][0]['rsad']['loc'], 'dt': '28/08/2023', 'asnm': inp['opdtls'][0]['pan']['num']}


In [148]:
a='''{"aplCd":"48950504","rgdtls":{"aplst":"DFT","aplty":"APLRG","duedt":"23/09/2023","moddt":"08/09/2023","mbno":"9481829401","em":"kashvijain19@gmail.com"},"asgdtls":[{"mn":"DILEEP","ln":" JAIN","mbno":"9481829401","fhmn":"KUMAR","gd":"F","fhln":"JAIN","iscitind":"Y","pan":{"num":"BUTPJ7345F"},"fhfn":"DILEEP","rsad":{"flno":"2nd Floor","stcd":"29","bno":"8/26","bnm":"8/26","loc":"Bengaluru","dst":"KABNU","st":"Chandra Shekhar Layout Road","pncd":"560004","cnty":"IND"},"dg":"SOLE PROPRIETOR","dob":"19/10/2000","dcupdtls":[{"ty":"PHOT","id":"107023630","ct":"image/jpeg","hash":"7167284b4987aaa4cf9a3f53c8bcf7723ad8bc0a4b04950710cf4deff688647a"}],"em":"kashvijain19@gmail.com","fn":"KASHVI","isasg":"Y","eid":"2","isNewSignatoryDetailsAfterSCN":false,"ispas":"Y","aadhaarflag":"Y"}],"bzdtls":{"bzdtlsbz":{"addltrdnms":[],"regCategory":[],"rslibrg":"DSEP","iscasdl":"N","isopcmp":"N","cobz":"PRO","lgnmbzpan":"KASHVI DILEEP JAIN","pan":{"num":"BUTPJ7345F","iv":"V","panDate":"Pan date not available"},"stcd":"29","trdnm":"highheelsnthreads","librgdt":"08/09/2023","cmbzdt":"08/09/2023","otherRsLib":"","dst":"KABNU","sezDtls":{}},"exrgdtls":[]},"opdtls":[{"mn":"DILEEP","ln":" JAIN","mbno":"9481829401","fhmn":"KUMAR","gd":"F","fhln":"JAIN","iscitind":"Y","pan":{"num":"BUTPJ7345F"},"fhfn":"DILEEP","rsad":{"flno":"2nd Floor","stcd":"29","bno":"8/26","bnm":"8/26","loc":"Bengaluru","dst":"KABNU","st":"Chandra Shekhar Layout Road","pncd":"560004","cnty":"IND"},"dg":"SOLE PROPRIETOR","dob":"19/10/2000","dcupdtls":[{"ty":"PHOT","id":"107023630","ct":"image/jpeg","hash":"7167284b4987aaa4cf9a3f53c8bcf7723ad8bc0a4b04950710cf4deff688647a"}],"em":"kashvijain19@gmail.com","fn":"KASHVI","isasg":"Y","eid":"1","isNewSignatoryDetailsAfterSCN":false}],"decdtls":{"dt":"08/09/2023"},"secToken":"292300186800TRN","map":{"asgdtls":[{"eid":"2","lt":"12.951735","lg":"77.568406","eloc":"E71XNI","city":"Bengaluru","geoCodeLevel":"Building","locSubloc":"Basavanagudi","feedback":{"locSubloc":false,"st":false,"bnm":false,"bno":false,"nrbyLdMrk":false}}],"opdtls":[{"eid":"1","lt":"12.951735","lg":"77.568406","eloc":"E71XNI","city":"Bengaluru","geoCodeLevel":"Building","locSubloc":"Basavanagudi","feedback":{"locSubloc":true,"st":true,"bnm":true,"bno":true,"nrbyLdMrk":false}}],"ppbzdtls":{"lt":"12.946982","lg":"77.5771940000001","eloc":"CX6X0C","city":"Bengaluru","geoCodeLevel":"Building","nrbyLdMrk":"Near jain bakes","locSubloc":"Basavanagudi","feedback":{"locSubloc":true,"st":true,"bnm":false,"bno":true,"nrbyLdMrk":false}}},"ppbzdtls":{"add":{"stcd":"29","pncd":"560004","dst":"KABNU","loc":"Bengaluru","lg":"77.5771940000001","lt":"12.946982","st":"Surveyor Street","bnm":"symbiosis padaki","bno":"14/1","flno":"5FC"},"ntbz":["RBU"],"contdtls":{"em":"kashvijain19@gmail.com","mbno":"9481829401"},"hasaddl":"N","sj":{"stjd":" LOCAL GST Office","stjdno":"KA024"},"cj":{"comrtcd":"YV","divsncd":"YV03","rangecd":"YV0301"},"psnt":"CON","dcupdtls":[{"id":"2023092940623","ty":"CNLR","ct":"application/pdf","docName":"noc.pdf","hash":"0c019b397837dee21ca7d579ffe1751bd95e47390bee2c008d6d0c32aab8eccd"},{"id":"2023092940627","ty":"ELCB","ct":"image/jpeg","docName":"WhatsApp_Image_2023_08_31_at_1.52.20_PM (1).jpeg","hash":"7167284b4987aaa4cf9a3f53c8bcf7723ad8bc0a4b04950710cf4deff688647a"}]},"gdssvcdtls":{"bzgddtls":[{"gdes":"Articles of apparel and clothing accessories","hsncd":"430310","eid":3}],"bzsdtls":[]},"stsidtls":{},"adhrflag":"Y"}'''

In [149]:
def find_nested_key_differences(dict1, dict2, parent_key=""):
    differences = {}

    for key in dict1.keys():
        if key not in dict2:
            differences[parent_key + '.' + key if parent_key else key] = ("Missing in dict2", dict1[key])
        elif isinstance(dict1[key], dict) and isinstance(dict2[key], dict):
            nested_diffs = find_nested_key_differences(dict1[key], dict2[key], key)
            differences.update(nested_diffs)
        elif dict1[key] != dict2[key]:
            differences[parent_key + '.' + key if parent_key else key] = (dict1[key], dict2[key])

    for key in dict2.keys():
        if key not in dict1:
            differences[parent_key + '.' + key if parent_key else key] = ("Missing in dict1", dict2[key])

    return differences


differences = find_nested_key_differences(json.loads(a), inp)
for key, value in differences.items():
    print(key, ":", value)

aplCd : ('48950504', 48952576)
asgdtls : ([{'mn': 'DILEEP', 'ln': ' JAIN', 'mbno': '9481829401', 'fhmn': 'KUMAR', 'gd': 'F', 'fhln': 'JAIN', 'iscitind': 'Y', 'pan': {'num': 'BUTPJ7345F'}, 'fhfn': 'DILEEP', 'rsad': {'flno': '2nd Floor', 'stcd': '29', 'bno': '8/26', 'bnm': '8/26', 'loc': 'Bengaluru', 'dst': 'KABNU', 'st': 'Chandra Shekhar Layout Road', 'pncd': '560004', 'cnty': 'IND'}, 'dg': 'SOLE PROPRIETOR', 'dob': '19/10/2000', 'dcupdtls': [{'ty': 'PHOT', 'id': '107023630', 'ct': 'image/jpeg', 'hash': '7167284b4987aaa4cf9a3f53c8bcf7723ad8bc0a4b04950710cf4deff688647a'}], 'em': 'kashvijain19@gmail.com', 'fn': 'KASHVI', 'isasg': 'Y', 'eid': '2', 'isNewSignatoryDetailsAfterSCN': False, 'ispas': 'Y', 'aadhaarflag': 'Y'}], [{'iscitind': 'Y', 'isNewSignatoryDetailsAfterSCN': False, 'fn': 'KASHVI', 'mn': 'DILEEP', 'ln': '  JAIN', 'pan': {'num': 'BUTPJ7345F'}, 'rsad': {'cnty': 'IND', 'pncd': '560004', 'dst': 'KABNU', 'loc': 'Bengaluru', 'stcd': '29', 'st': 'Chandra Shekhar Layout Road', 'bnm':

In [150]:
inp['asgdtls']

[{'iscitind': 'Y',
  'isNewSignatoryDetailsAfterSCN': False,
  'fn': 'KASHVI',
  'mn': 'DILEEP',
  'ln': '  JAIN',
  'pan': {'num': 'BUTPJ7345F'},
  'rsad': {'cnty': 'IND',
   'pncd': '560004',
   'dst': 'KABNU',
   'loc': 'Bengaluru',
   'stcd': '29',
   'st': 'Chandra Shekhar Layout Road',
   'bnm': '8/26',
   'bno': '8/26',
   'flno': '2nd Floor'},
  'isasg': 'Y',
  'fhfn': 'DILEEP',
  'fhmn': 'KUMAR',
  'fhln': 'JAIN',
  'dob': '19/10/2000',
  'mbno': '9481829401',
  'em': 'kashvijain19@gmail.com',
  'gd': 'F',
  'dg': 'SOLE PROPRIETOR',
  'dcupdtls': [{'id': 107027179,
    'ty': 'PHOT',
    'ct': 'image/jpeg',
    'hash': 'ccb45941ebdce11e6d5ef25b23c247d21551be69a658f271ce7f1013fde56616'}],
  'eid': 2,
  'ispas': 'Y',
  'aadhaarflag': 'Y'}]

In [151]:
json.dumps(inp)

'{"aplCd": 48952576, "rgdtls": {"aplst": "DFT", "aplty": "APLRG", "duedt": "23/09/2023", "moddt": "08/09/2023", "mbno": "9481829401", "em": "kashvijain19@gmail.com"}, "bzdtls": {"bzdtlsbz": {"addltrdnms": [], "regCategory": [], "rslibrg": "DSEP", "iscasdl": "N", "isopcmp": "N", "lgnmbzpan": "KASHVI DILEEP  JAIN", "pan": {"num": "BUTPJ7345F", "iv": "V", "panDate": "Pan date not available"}, "stcd": "29", "dst": "KABNU", "trdnm": "highheelsnthreads", "cobz": "PRO", "cmbzdt": "08/09/2023", "librgdt": "08/09/2023", "otherRsLib": "", "sezDtls": {}}, "exrgdtls": []}, "decdtls": {"asdes": "SOLE PROPRIETOR", "pl": "Bengaluru", "dt": "28/08/2023", "asnm": "BUTPJ7345F"}, "secToken": "292300186899TRN", "map": {"opdtls": [{"eid": 1, "lt": "12.951735", "lg": "77.568406", "eloc": "E71XNI", "city": "Bengaluru", "geoCodeLevel": "Building", "nrbyLdMrk": null, "locSubloc": "Sankarapuram", "feedback": {"locSubloc": true, "st": true, "bnm": true, "bno": true, "nrbyLdMrk": false}}], "asgdtls": [{"eid": 2, 

In [152]:
url = "https://reg.gst.gov.in/registration/auth/api/save/appdraftnewreg"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": new_cookie,
    "Content-Type": "application/json;charset=UTF-8",
    "At": auth_token,
    "Origin": "https://reg.gst.gov.in",
   }
response = requests.post(url, headers=headers, data=json.dumps(inp))
print(response.text)



{"aplId":"48952576","stscd":"SUCCESS","errors":""}


In [153]:
ck=response.headers['Set-Cookie'].split(',')[-1].split(';')[0]

In [154]:
new_cookie1 = f'AuthToken={auth_token};{ck}'

In [155]:
trn

'292300186899TRN'

In [156]:
import requests

url = 'https://reg.gst.gov.in/registration/auth/api/evc/otp'

headers = {
    'Host': 'reg.gst.gov.in',
    'Cookie': new_cookie1,
    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/json;charset=UTF-8',
    'At': auth_token,
    'Origin': 'https://reg.gst.gov.in',
    'Referer': 'https://reg.gst.gov.in/registration/auth/newappl/verify',
    }

data = {
    "formType":"APLRG","tranId":trn,"userName":inp['opdtls'][0]['fn']+" "+inp['opdtls'][0]['mn']+" "+inp['opdtls'][0]['ln'],"email":inp['opdtls'][0]['em'],"mobNo":inp['opdtls'][0]['mbno']
}

response = requests.post(url, json=data, headers=headers)
print("Response Content:", response.content)
tranid=json.loads(response.text)['message']


Response Content: b'{"stscd":"1","message":"ca2a6a8dc1604905b22e7e70dad79f17","errors":""}'


In [157]:
trn

'292300186899TRN'

In [158]:
otp="7855E2"

In [159]:
import requests

url = 'https://reg.gst.gov.in/registration/auth/api/evc/submit'
headers = {
    'Host': 'reg.gst.gov.in',
    'Cookie': new_cookie,
    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/json;charset=UTF-8',
    'At': auth_token,
    'Origin': 'https://reg.gst.gov.in',
    'Referer': 'https://reg.gst.gov.in/registration/auth/newappl/verify',
    }
data = {
    "otp": otp,
    "dataToSign": json.dumps(inp),
    "formType": "APLRG",
    "tranId": tranid,
    "userId": trn
}

response = requests.post(url, json=data, headers=headers)
print("Response Content:", response.content)


Response Content: b'{"stscd":"1","message":"SUCCESS","errors":""}'


In [162]:
trn

'292300186899TRN'